In [145]:
from scipy.spatial.transform import Rotation
from scipy.linalg import expm
import numpy as np
from Functions import inputdata, get_abaqus_data
from pyquaternion import Quaternion

In [146]:
# v_data = get_abaqus_data('test05V.txt', 265, 23)
def make_amp(v_data, delta_t=0.001, init_v=[-0.22906634372358425, 0.07038992291148816, -0.47632976594513821, 1.56221576610329290, 2.14566704525587680, -0.07210294407980200]):
    step = v_data.shape[0]
    npoin = v_data.shape[1] // 6
    amp_labels = ['VX', 'VY', 'VZ', 'RVX', 'RVY', 'RVZ']
    for index, label in enumerate(amp_labels):
        with open('amp_{0}_{1}step_{2}point'.format(label, step, npoin), 'w', encoding='ascii') as f:
            f.write('0 {0}\n'.format(init_v[index]))
            for s in range(1, step):
                f.write('{0:.3f} {1}\n'.format(delta_t*s, v_data[s, index]))                

In [147]:
def show(ax, ay, az):
    for i, j, k in zip(ax, ay, az):
        print('{0:.3f} {1:.5f} {2:.5f} {3:.5f}'.format(i[0], i[1], j[1], k[1]))

In [114]:
def make_O(wx, wy, wz):
    return np.array([[0, wz, -wy, wx],
                           [-wz, 0, wx, wy],
                           [wy, -wx, 0, wz],
                           [-wx, -wy, -wz, 0]], dtype=np.float64) 

In [115]:
def norm2(v):
    return np.sqrt(np.sum(v**2))

def get_q0(originCoordsG):
    npoin = originCoordsG.shape[1]
    a = np.array([originCoordsG[0, npoin-1], originCoordsG[1, npoin-1], originCoordsG[2, npoin-1]], 
                dtype=np.float64)
    b = a / norm2(a)
    x, y, z = b[0], b[1], b[2]
    
    theta1 = -np.arctan2(y, x)
    tmp_q1 = Quaternion(axis=np.array([0, 0, 1]), angle=theta1)
    
    c = tmp_q1.rotate(b)
#     print(c)   
    
    x1, y1, z1 = c[0], c[1], c[2]
    theta2 = np.arctan2(z1, x1)
    tmp_q2 = Quaternion(axis=np.array([0, 1, 0]), angle=theta2)
    
#     d = tmp_q2.rotate(c)
#     print(d)
    
    q0 = tmp_q2 * tmp_q1    
#     e = q0.rotate(b)
#     print(e)

    return q0
    



In [137]:
def make_rig_RA_RU(VX, VY, VZ, RVX, RVY, RVZ, delta_t ,originCoordsG):    
    n_t = VX.shape[0] - 1
    npoin = originCoordsG.shape[1]
    # 原点情報    
    UX = np.zeros((n_t+1, 2), dtype=np.float64)
    UY = np.zeros((n_t+1, 2), dtype=np.float64)
    UZ = np.zeros((n_t+1, 2), dtype=np.float64)    
    AX = np.zeros((n_t+1, 2), dtype=np.float64)
    AY = np.zeros((n_t+1, 2), dtype=np.float64)    
    AZ = np.zeros((n_t+1, 2), dtype=np.float64)           
    
    RAX = np.zeros((n_t+1, 2), dtype=np.float64)
    RAY = np.zeros((n_t+1, 2), dtype=np.float64)    
    RAZ = np.zeros((n_t+1, 2), dtype=np.float64)

    for i in range(1, n_t+1):
        # 時間設定
        UX[i, 0] = VX[i, 0]
        AX[i, 0] = VX[i, 0]            
        UY[i, 0] = VY[i, 0]
        AY[i, 0] = VY[i, 0]
        UZ[i, 0] = VZ[i, 0]
        AZ[i, 0] = VZ[i, 0]
        
        RAX[i, 0] = RVX[i, 0]
        RAY[i, 0] = RVY[i, 0]
        RAZ[i, 0] = RVZ[i, 0]
        
        if i > 1:
            UX[i, 1] = (VX[i, 1] + VX[i-1, 1])/2 * delta_t + UX[i-1, 1]
            UY[i, 1] = (VY[i, 1] + VY[i-1, 1])/2 * delta_t + UY[i-1, 1] 
            UZ[i, 1] = (VZ[i, 1] + VZ[i-1, 1])/2 * delta_t + UZ[i-1, 1]
                        
            if i < n_t:
                AX[i, 1] = (VX[i+1, 1]-VX[i-1, 1])/2/delta_t
                AY[i, 1] = (VY[i+1, 1]-VY[i-1, 1])/2/delta_t  
                AZ[i, 1] = (VZ[i+1, 1]-VZ[i-1, 1])/2/delta_t                
                
                RAX[i, 1] = (RVX[i+1, 1]-RVX[i-1, 1])/2/delta_t
                RAY[i, 1] = (RVY[i+1, 1]-RVY[i-1, 1])/2/delta_t                 
                RAZ[i, 1] = (RVZ[i+1, 1]-RVZ[i-1, 1])/2/delta_t 
                
    # 初期設定
    AX[1, 1] = AX[2, 1]
    AY[1, 1] = AY[2, 1]
    AZ[1, 1] = AZ[2, 1]
    
    RAX[1, 1] = RAX[2, 1]
    RAY[1, 1] = RAY[2, 1]
    RAZ[1, 1] = RAZ[2, 1]
    
    tmp = np.array([originCoordsG[0, npoin-1], originCoordsG[1, npoin-1], originCoordsG[2, npoin-1]], dtype=np.float64)
    originE =  (1/np.sum(np.sqrt(tmp**2)))*tmp
    q0 = Quaternion(np.array([1, 0, 0, 0]))
#     q0 = get_q0(originCoordsG)
    qs = [q0]
    total_qs = [q0]
    
    for step in range(1, n_t+1): # step-1からstepを作る
        wx = RVX[step, 1]
        wy = RVY[step ,1]
        wz = RVZ[step, 1]
#         O = make_W(RVX[step, 1], RVY[step, 1], RVZ[step, 1])        
#         P = expm(1/2*delta_t*O)
        
        q_prev = total_qs[step-1]
#         print(delta_t, wx, wy, wz)
        q_next = Quaternion()    
#         print(wx, wy, wz)
        q_next.integrate(np.array([wx, wy, wz], dtype=np.float64), delta_t)
    
        qs.append(q_next)
#         print(q_next, q_next*q_prev)
        total_qs.append(q_next*q_prev)
        
    return UX, UY, UZ, AX, AY, AZ, RAX, RAY, RAZ, qs, total_qs    

In [240]:
# UX, UY, UZ, AX, AY, AZ, RAX, RAY, RAZ, qs, total_qs = make_rig_RA_RU(VX, VY, VZ, RVX, RVY, RVZ, delta_t ,originCoordsG)

In [241]:
# for index, q in enumerate(total_qs):
#     ox = originCoordsG[0, npoin-1]
#     oy = originCoordsG[1, npoin-1]
#     oz = originCoordsG[2, npoin-1]
#     ux = UX[index+1, 1]
#     uy = UY[index+1, 1]
#     uz = UZ[index+1, 1]    
#     c = q.rotate(np.array([ox, oy, oz]))
#     print(index, c[0]+ux-ox, c[1]+uy-oy, c[2]+uz-oz)

In [242]:
# inp_path = 'test08.txt'
# originCoordsG, ae, npoin, nele, delta_t, n_t, VX, VY, VZ, RVX, RVY, RVZ, head_mass, gamma, omega, HHT_alpha \
# = inputdata(inp_path)

In [243]:
# show(VX, VY, VZ)

In [244]:
def make_expanded_amp(V_array, rate):    
    labels = ['VX', 'VY', 'VZ', 'RVX', 'RVY', 'RVZ']
    for V, label in zip(V_array, labels):
        delta_t = V[2, 0]
        size = len(V)-1
        expanded_amp = np.zeros((size*rate+1, 2), dtype=np.float64)
#         print(expanded_amp[0, 0], expanded_amp[0, 1])
        for i in range(size):
            for idx, j in enumerate(range(i*rate+1, (i+1)*rate+1)):                
                expanded_amp[j-1, 0] = (j-1) * delta_t
                expanded_amp[j-1, 1] = (V[i,1] + (V[i+1, 1]-V[i,1])/rate*(idx+1))/rate
#                 print(i, j, expanded_amp[j-1, 0], expanded_amp[j-1, 1])
#         return
        with open('amp_{0}_expand{1}.txt'.format(label, rate), 'w', encoding='ascii') as f:
            for amp in expanded_amp[:-1]:
                if i < len(expanded_amp[:-1])-1:
                    f.write('{0:.5f} {1}\n'.format(amp[0], amp[1]))
                else:
                    f.write('{0:.5f} {1}'.format(amp[0], amp[1]))

In [245]:
# make_expanded_amp([VX, VY, VZ, RVX, RVY, RVZ], 10)
# make_expanded_amp([VX, VY, VZ, RVX, RVY, RVZ], 5)
# make_expanded_amp([VX, VY, VZ, RVX, RVY, RVZ], 3)
# make_expanded_amp([VX, VY, VZ, RVX, RVY, RVZ], 2)

In [248]:
# show(RVX, RVY, RVZ)

0.000 0.00000 0.00000 0.00000
0.000 1.56222 2.14567 -0.07210
0.001 1.54726 2.14671 -0.05277
0.002 1.53231 2.14776 -0.03345
0.003 1.48071 2.15138 0.03326
0.004 1.37168 2.15902 0.17421
0.005 1.25794 2.16700 0.32135
0.006 1.13650 2.17568 0.47981
0.007 1.01004 2.18480 0.64567
0.008 0.88677 2.19383 0.80846
0.009 0.77658 2.20242 0.95872
0.010 0.66976 2.21090 1.10577
0.011 0.55908 2.21977 1.25891
0.012 0.44221 2.22928 1.42182
0.013 0.32563 2.23879 1.58457
0.014 0.22032 2.24761 1.73368
0.015 0.12255 2.25597 1.87366
0.016 0.02353 2.26470 2.01551
0.017 -0.08082 2.27496 2.16523
0.018 -0.18656 2.28562 2.31702
0.019 -0.28606 2.29627 2.46000
0.020 -0.37529 2.30690 2.58850
0.021 -0.46382 2.31792 2.71572
0.022 -0.55393 2.33508 2.84085
0.023 -0.64513 2.35646 2.96455
0.024 -0.73381 2.37836 3.08403
0.025 -0.81165 2.40253 3.18529
0.026 -0.88658 2.42731 3.28164
0.027 -0.96139 2.45751 3.37318
0.028 -1.03603 2.49683 3.45665
0.029 -1.11021 2.53690 3.53878
0.030 -1.17726 2.57690 3.60979
0.031 -1.23931 2.61684 

In [251]:
inp_path = 'beam-average2.txt'
originCoordsG2, ae2, npoin2, nele2, delta_t2, n_t2, VX2, VY2, VZ2, RVX2, RVY2, RVZ2, head_mass2, gamma2, omega2, HHT_alpha2 \
= inputdata(inp_path)

In [252]:
UX2, UY2, UZ2, AX2, AY2, AZ2, RAX2, RAY2, RAZ2, qs2, total_qs2 = \
make_rig_RA_RU(VX2, VY2, VZ2, RVX2, RVY2, RVZ2, delta_t2, originCoordsG2)

In [254]:
show(RAX2, RAY2, RAZ2)

0.000 0.00000 0.00000 0.00000
0.000 193.40750 268.33875 -6.59680
0.001 193.40750 268.33875 -6.59680
0.002 -3.73894 0.26074 4.83214
0.003 -3.73822 0.26162 4.83213
0.004 -3.73750 0.26250 4.83212
0.005 -8.31875 0.58375 10.75463
0.006 -12.90000 0.90500 16.67713
0.007 -20.07875 1.40750 25.95736
0.008 -27.25750 1.91000 35.23760
0.009 -27.84625 1.95250 36.01117
0.010 -28.43500 1.99500 36.78475
0.011 -29.39750 2.08250 38.19950
0.012 -30.36000 2.17000 39.61425
0.013 -30.98750 2.22500 40.53925
0.014 -31.61500 2.28000 41.46425
0.015 -31.21687 2.26875 41.08163
0.016 -30.81875 2.25750 40.69900
0.017 -29.18225 2.20250 39.13225
0.018 -27.54575 2.14750 37.56550
0.019 -27.12562 2.13375 37.16350
0.020 -26.70550 2.12000 36.76150
0.021 -27.18750 2.16875 37.52325
0.022 -27.66950 2.21750 38.28500
0.023 -28.44325 2.29750 39.50625
0.024 -29.21700 2.37750 40.72750
0.025 -29.18162 2.37750 40.70750
0.026 -29.14625 2.37750 40.68750
0.027 -27.73688 2.29125 38.98250
0.028 -26.32750 2.20500 37.27750
0.029 -25.38437 